# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
customer_purchases = data.groupby(["CustomerID","ProductName"], as_index = False).agg({"Quantity": "sum"})

In [5]:
customer_purchases.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
purchase_qty = pd.pivot_table(data, values='Quantity', index='ProductName', columns='CustomerID',fill_value=0)
purchase_qty.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
customer_similarity = squareform(pdist(purchase_qty.T, 'euclidean'))

In [8]:
customer_similarity[:5]

array([[  0.        ,  10.72380529,   9.64365076, ..., 197.94948851,
        218.8241303 , 203.79892051],
       [ 10.72380529,   0.        ,  10.67707825, ..., 197.34994299,
        218.62525014, 203.95342606],
       [  9.64365076,  10.67707825,   0.        , ..., 197.32460566,
        218.02981447, 203.80628057],
       [  9.89949494,  10.72380529,  10.34408043, ..., 197.32207175,
        218.14215549, 203.68112333],
       [ 10.44030651,  11.48912529,  10.19803903, ..., 197.46645285,
        217.92888748, 202.96058731]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [9]:
similarity = 1/(1+customer_similarity) 
top5_similarity = pd.DataFrame(similarity, index = purchase_qty.columns, columns = purchase_qty.columns)

In [10]:
top5_similarity.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


In [11]:
top_clients = pd.DataFrame(customer_similarity, index = purchase_qty.columns, columns = purchase_qty.columns)
top5 = top_clients[200].sort_values()[1:6]
top5

CustomerID
1920     9.591663
1072     9.797959
3317    10.099505
3253    10.099505
3909    10.148892
Name: 200, dtype: float64

In [12]:
top_5 = top5_similarity[200].sort_values(ascending=False)[1:6]
top_5

CustomerID
1920    0.094414
1072    0.092610
3253    0.090094
3317    0.090094
3909    0.089695
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [13]:
records = list(top_5.index)

In [14]:
total_records = customer_purchases[customer_purchases["CustomerID"].isin(records)].reset_index(drop=True)

In [15]:
total_records

,CustomerID,ProductName,Quantity
0,1072,"Appetizer - Mini Egg Roll, Shrimp",1
1,1072,Appetizer - Sausage Rolls,1
2,1072,"Beans - Kidney, Canned",1
3,1072,Beef - Montreal Smoked Brisket,1
4,1072,Beef - Striploin Aa,1
...,...,...,...
256,3909,Wine - Charddonnay Errazuriz,2
257,3909,Wine - Fume Blanc Fetzer,1
258,3909,"Wine - Magnotta, Merlot Sr Vqa",1
259,3909,Wine - Toasted Head,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [42]:
total_records.reset_index(drop=False,inplace=True)

In [43]:
aggr = total_records.groupby('ProductName',as_index=False).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).reset_index(drop=True)

In [44]:
aggr

,ProductName,Quantity
0,Soup - Campbells Bean Medley,3
1,Lamb - Ground,3
2,Pomello,3
3,Bay Leaf,3
4,Pork - Kidney,3
...,...,...
212,Kiwi,1
213,Knife Plastic - White,1
214,"Lamb - Pieces, Diced",1
215,Lambcasing,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [19]:
chosen = aggr.merge(purchase_qty, on='ProductName')

filtro = chosen[chosen[200] == 0]
filtro.sort_values("Quantity",ascending=False,inplace=True)
recomendaciones = list(filtro[:5]["ProductName"])
recomendaciones

/Users/Menazina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


['Soup - Campbells Bean Medley',
 'Bay Leaf',
 'Pork - Kidney',
 'Muffin - Carrot Individual Wrap',
 'Scampi Tail']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [24]:
recommendation = {}
clients = list(data['CustomerID'].unique())

In [25]:
for j in clients:
    top5 = top_clients[j].sort_values()[1:5]
    topi5 = list(top5.index)
    records = customer_purchases.loc[topi5,:]
    records.reset_index(drop=False,inplace=True)
    pro2 = records.groupby('ProductName').agg({'Quantity':'sum'})
    pro2.sort_values('Quantity',ascending=False,inplace=True)
    matrix = pro2.merge(purchase_qty[j],on='ProductName')
    matrix = matrix[matrix[j]==0]
    matrix.sort_values('Quantity',ascending=False,inplace=True)
    recommendation[j]=list(matrix[:5].index)

In [26]:
recommendation[33]

['Cinnamon Buns Sticky',
 'Skirt - 29 Foot',
 'Bread - Italian Corn Meal Poly',
 'Cheese - Camembert']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [32]:
rec = pd.DataFrame(recomendaciones).T
rec.columns = ['Recommendation1','Recommendation2','Recommendation3','Recommendation4','Recommendation5']
rec

,Recommendation1,Recommendation2,Recommendation3,Recommendation4,Recommendation5
0,Soup - Campbells Bean Medley,Bay Leaf,Pork - Kidney,Muffin - Carrot Individual Wrap,Scampi Tail


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [33]:
customer_sim_2 = squareform(pdist(purchase_qty.T, 'jaccard'))
top_2 = pd.DataFrame(customer_sim_2, index = purchase_qty.columns, columns = purchase_qty.columns)

In [34]:
top_2[:5]

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,0.934959,0.869159,0.899083,0.923729,0.919643,0.950413,0.945946,0.938596,0.927928,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
200,0.934959,0.000000,0.919355,0.927419,0.970588,0.953125,0.906250,0.960000,0.919355,0.909091,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
264,0.869159,0.919355,0.000000,0.930435,0.888889,0.940171,0.934426,0.965517,0.940171,0.948276,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
356,0.899083,0.927419,0.930435,0.000000,0.933884,0.948718,0.933884,0.908257,0.957627,0.947826,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
412,0.923729,0.970588,0.888889,0.933884,0.000000,0.934426,0.912000,0.904348,0.943089,0.915254,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [35]:
recommendation_2 = {}

In [41]:
for j in clients:
    top_five = top_2[j].sort_values()[1:6]
    top_f = list(top_five.index)
    records2 = customer_purchases.loc[top_f,:]
    records2.reset_index(drop=False,inplace=True)
    pro3 = records2.groupby('ProductName').agg({'Quantity':'sum'})
    pro3.sort_values('Quantity',ascending=False,inplace=True)
    matrix2 = pro3.merge(purchase_qty[j],on='ProductName')
    matrix2 = matrix2[matrix2[j]==0]
    matrix2.sort_values('Quantity',ascending=False,inplace=True)
    recommendation_2[j]=list(matrix2[:5].index)

KeyError: "None of [Int64Index([78048, 77810, 75793, 77651, 75762], dtype='int64')] are in the [index]"

In [ ]:
#For some reason the index doesnt work as i plenes, Im trying to drop the index since the beggining but it keep bringing me the original index.